# Inicialización Grader


In [2]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


# Código

Código correspondiente a la carga, limpieza y preprocesado de los datos de entrenamiento del modelo.

## Archivos de datos

In [8]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.05GB/s]


Se descomprime el archivo con los datos:

In [9]:
!unzip udea*.zip > /dev/null

Se hace un recuento preliminar del tamaño de los archivos:

In [10]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


## Librerías

In [11]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Carga de datos

In [58]:
training_data = pd.read_csv("train.csv")
print ("Tamaño del DataFrame:", training_data.shape)

Tamaño del DataFrame: (692500, 21)


Breve exploración de los datos y las columnas:

In [59]:
training_data.head(10)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,medio-alto,0.553,0.142,0.248,0.175
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Si,...,N,Si,Si,Si,Técnica o tecnológica completa,alto,0.242,0.299,0.267,0.308
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Si,...,N,Si,No,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.238,0.316,0.286,0.290
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Educación profesional incompleta,medio-bajo,0.147,0.407,0.212,0.328
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Si,...,N,Si,Si,Si,Postgrado,alto,0.535,0.122,0.257,0.167


Nombres de las variables/atributos/columnas

In [60]:
training_data.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

Tipos de datos de cada columna:

In [61]:
for c in training_data.columns:
    print ("%20s"%c, training_data[c].dtype)

                  ID int64
   PERIODO_ACADEMICO int64
    E_PRGM_ACADEMICO object
 E_PRGM_DEPARTAMENTO object
E_VALORMATRICULAUNIVERSIDAD object
E_HORASSEMANATRABAJA object
   F_ESTRATOVIVIENDA object
     F_TIENEINTERNET object
    F_EDUCACIONPADRE object
     F_TIENELAVADORA object
    F_TIENEAUTOMOVIL object
  E_PRIVADO_LIBERTAD object
E_PAGOMATRICULAPROPIO object
   F_TIENECOMPUTADOR object
   F_TIENEINTERNET.1 object
    F_EDUCACIONMADRE object
  RENDIMIENTO_GLOBAL object
         INDICADOR_1 float64
         INDICADOR_2 float64
         INDICADOR_3 float64
         INDICADOR_4 float64


Podemos ver que únicamente las columnas de indicadores estadísticos del 1-4 representan variables continuas, mientras que el resto representan variables categóricas o intervalos.

Si bien la variable "Período Académico" se representa por un número, sabemos que en realidad este número se asocia a un indicador temporal (semestre) y no a una variable continua propiamente.

Por otro lado, la columna ID no representa información relevante pues es únicamente un identificador de cada estudiante.

## Valores faltantes o nulos

Cantidades por columna:

In [70]:
missing_values = training_data.isna().sum()
missing_values[missing_values!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


Columnas que tienen valores nulos:

In [73]:
missing_values[missing_values!=0].index

Index(['E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PAGOMATRICULAPROPIO',
       'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1', 'F_EDUCACIONMADRE'],
      dtype='object')

In [74]:
for c in missing_values[missing_values!=0].index:
    print ("%20s"%c, training_data[c].dtype)

E_VALORMATRICULAUNIVERSIDAD object
E_HORASSEMANATRABAJA object
   F_ESTRATOVIVIENDA object
     F_TIENEINTERNET object
    F_EDUCACIONPADRE object
     F_TIENELAVADORA object
    F_TIENEAUTOMOVIL object
E_PAGOMATRICULAPROPIO object
   F_TIENECOMPUTADOR object
   F_TIENEINTERNET.1 object
    F_EDUCACIONMADRE object


Podemos ver que todas las columnas con valores nulos son de tipo categórico

In [75]:
print(f"Filas con valores nulos: {missing_values[missing_values!=0].shape[0]}")
print(f"Atributo con mayor número de valores nulos: {missing_values[missing_values!=0].sort_values(ascending=False).index[0]}, con un total de {max(missing_values[missing_values!=0])}")

Filas con valores nulos: 11
Atributo con mayor número de valores nulos: F_TIENEAUTOMOVIL, con un total de 43623


Podemos observar que de las 21 columnas presentes en el modelo, 11 de ellas presentan valores nulos y la columna con mayor cantidad de valores nulos es "F_TIENEAUTOMOVIL" con un total de 43623 registros cuyo valor es nulo, frente a 692500 registros totales, aproximadamente un 6% de los datos totales.

## Variables categóricas

¿Cuáles variables son exclusivamente de tipo categórico?

In [77]:
categorical_columns = [i for i in training_data.columns if not i in training_data._get_numeric_data()]
categorical_columns

['E_PRGM_ACADEMICO',
 'E_PRGM_DEPARTAMENTO',
 'E_VALORMATRICULAUNIVERSIDAD',
 'E_HORASSEMANATRABAJA',
 'F_ESTRATOVIVIENDA',
 'F_TIENEINTERNET',
 'F_EDUCACIONPADRE',
 'F_TIENELAVADORA',
 'F_TIENEAUTOMOVIL',
 'E_PRIVADO_LIBERTAD',
 'E_PAGOMATRICULAPROPIO',
 'F_TIENECOMPUTADOR',
 'F_TIENEINTERNET.1',
 'F_EDUCACIONMADRE',
 'RENDIMIENTO_GLOBAL']

Listado de valores posibles iniciales (no nulos):

In [78]:
for c in categorical_columns:
    print("%10s"%c, np.unique(training_data[c].dropna()))

E_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GESTISN 

## Eliminación de valores nulos

Vamos a asignar a todos los valores nulos el valor "Sin especificar" de modo que sea más fácil realizar la conversión a variable numérica.

Dado que no hay variables nulas dentro de las columnas numéricas, podemos ejecutar una función única para todas las columnas con valores nulos

In [79]:
def eliminacion_nulos(column_name):
    training_data[column_name].values[training_data[column_name].isna()] = "Sin especificar"

for c in categorical_columns:
    eliminacion_nulos(c)

Validación de valores nulos después de hacer el reemplazo:

In [80]:
missing_values_after = training_data.isna().sum()
missing_values_after[missing_values_after!=0]


,0


In [81]:
training_data.head(10)

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,medio-alto,0.553,0.142,0.248,0.175
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Si,...,N,Si,Si,Si,Técnica o tecnológica completa,alto,0.242,0.299,0.267,0.308
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Si,...,N,Si,No,Si,Secundaria (Bachillerato) incompleta,medio-bajo,0.238,0.316,0.286,0.290
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Si,...,N,Si,Si,Si,Educación profesional incompleta,medio-bajo,0.147,0.407,0.212,0.328
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Si,...,N,Si,Si,Si,Postgrado,alto,0.535,0.122,0.257,0.167
